In [ ]:
import pandas as pd
import numpy as np

def calculate_optimal_pnl(df, heat_rate=10, mw=250, min_run_hours=3, min_down_hours=6):
    # Calculate potential PnL for each hour
    df['potential_pnl'] = (df['power_price'] - (df['gas_price'] * heat_rate)) * mw
    
    n = len(df)
    # Initialize columns for decisions and PnL
    df['run_decision'] = False
    df['pnl'] = 0.0
    
    # Dynamic programming to find the optimal operation schedule
    dp = [0] * (n + 1)  # DP array to store maximum PnL at each point considering constraints
    decision = [0] * n  # To track the operation decision and duration

    # Fill DP array from the end to the start
    for i in range(n - 1, -1, -1):
        # Scenario 1: Not running
        dp[i] = dp[i + 1]
        decision[i] = 0  # 0 hours of operation

        # Scenario 2: Running for at least min_run_hours and considering min_down_hours
        running_profit = 0
        for j in range(i, min(i + min_run_hours, n)):
            running_profit += df.iloc[j]['potential_pnl']
            if j + min_down_hours < n:  # Check if there's enough room for the minimum down time
                future_profit = dp[j + min_down_hours + 1] if (j + min_down_hours + 1 < n) else 0
                if running_profit + future_profit > dp[i]:
                    dp[i] = running_profit + future_profit
                    decision[i] = j - i + 1  # Update decision to run for these many hours

        # Extend running beyond min_run_hours if profitable
        for j in range(i + min_run_hours, n):
            running_profit += df.iloc[j]['potential_pnl']
            if j + 1 < n and j + min_down_hours < n:  # Ensure we can also have down time
                if running_profit + dp[j + min_down_hours + 1] > dp[i]:
                    dp[i] = running_profit + dp[j + min_down_hours + 1]
                    decision[i] = j - i + 1

    # Apply decisions to calculate PnL and run_decision
    i = 0
    while i < n:
        if decision[i] > 0:  # Plant decided to run
            for j in range(i, i + decision[i]):
                df.at[df.index[j], 'pnl'] = df.iloc[j]['potential_pnl']
                df.at[df.index[j], 'run_decision'] = True
            i += decision[i] + min_down_hours  # Skip the down time after running
        else:
            i += 1

    return df

# Example DataFrame for demonstration
data = {
    'time': pd.date_range(start='2022-01-01', periods=24, freq='H'),
    'power_price': np.random.uniform(50, 150, 24),  # Simulated power prices
    'gas_price': np.random.uniform(3, 10, 24),      # Simulated gas prices
}
df = pd.DataFrame(data)
df.set_index('time', inplace=True)

# Call the function
optimized_df = calculate_optimal_pnl(df)

# Display the optimized DataFrame
print(optimized_df[['power_price', 'gas_price', 'potential_pnl', 'run_decision', 'pnl']])

